In [39]:
import os
import numpy as np
from collections import Counter

In [40]:
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]
    print(len(emails))
    all_words = []
    for mail in emails:
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    dictionary_temp = Counter(all_words)

    # Paste code for non-word removal here(code snippet is given below)
    list_to_remove = dictionary_temp.keys()
    for item in list_to_remove:
        if item.isalpha() == False:    #Determine whether it is punctuation
            del dictionary[item]
        elif len(item) == 1:           #
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    return dictionary

In [41]:
def extract_features(root_dir):
    emails = [os.path.join(root_dir,f) for f in os.path(root_dir)]
    all_words = []
    features_matrix = np.zeros((len(emails),3000))
    for main in emails:
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:
                    words = line.split()
                    all_words += words
    for word in all_words:
        wordID = 0
        for i,d in enumerate(dictionary):
            if d[0] == word:
                wordID = i
                feat



In [42]:
Path = "./test-mails"
dir = make_Dictionary(Path)
print(len(dir))
Path = "./test-mails"
# features_matrix,labels = extract_features(Path)

260
3000
